In [9]:
import pandas as pd
import glob
import os

# 1. Configuración de rutas
# Buscamos todos los archivos .csv dentro de 'scraps' y sus subcarpetas
path_pattern = os.path.join('scraps', '**', '*.csv')
archivos = glob.glob(path_pattern, recursive=True)

print(f"Se han encontrado {len(archivos)} archivos para procesar.")

# 2. Lectura y unión de archivos
lista_df = []

for archivo in archivos:
    try:
        # Cargamos cada CSV
        temp_df = pd.read_csv(archivo)
        
        # Opcional: Añadir columna con el nombre del archivo de origen para trazabilidad
        ##temp_df['source_file'] = os.path.basename(archivo)
        
        lista_df.append(temp_df)
    except Exception as e:
        print(f"Error al leer {archivo}: {e}")

# Unimos todos en un solo DataFrame
if lista_df:
    df_maestro = pd.concat(lista_df, ignore_index=True)
    
    # 3. Limpieza de Duplicados
    # El 'id' o la 'job_url' son los identificadores únicos más fiables
    total_antes = len(df_maestro)
    
    # Eliminamos duplicados basados en la URL de la oferta
    df_maestro = df_maestro.drop_duplicates(subset=['job_url'], keep='first')

    # 2. Eliminamos duplicados que tengan el mismo título en la misma empresa y ciudad
    df_maestro.drop_duplicates(
        subset=['title', 'company', 'location'], 
        keep='first', # Nos quedamos con la primera que encontró
        inplace=True
    )
    
    total_despues = len(df_maestro)
    
    print("-" * 30)
    print(f"Total de ofertas leídas: {total_antes}")
    print(f"Ofertas únicas conservadas: {total_despues}")
    print(f"Duplicados eliminados: {total_antes - total_despues}")
    print("-" * 30)

    # 4. Guardar el Dataset Maestro
    # Lo guardamos fuera de la carpeta scraps para no volverlo a leer accidentalmente después
    nombre="dataset_maestro.csv"
    ruta_completa = os.path.join('data', nombre)
    df_maestro.to_csv(ruta_completa, index=False, encoding='utf-8')
    print(f"¡Dataset Maestro guardado con éxito como '{nombre}'!")
else:
    print("No se encontraron archivos para unir.")

Se han encontrado 2 archivos para procesar.
------------------------------
Total de ofertas leídas: 1296
Ofertas únicas conservadas: 1273
Duplicados eliminados: 23
------------------------------
¡Dataset Maestro guardado con éxito como 'dataset_maestro.csv'!
